In [ ]:
# librerias que vamos a usar

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# aca importamos todas las librerías que vamos a usar

# cargamso el dataset, siendo asi elmismoq ue usamos para el tp 1.
data = pd.read_csv("bank.csv")

# mostrar las primeras filas
data.head()

In [ ]:
# etapa de preprocesamiento de los datos, para que puedan ser usados posteriormente

# columnas con texto, categoricas
columnas_categoricas = ['job', 'marital', 'education', 'default', 'housing',
                        'loan', 'contact', 'month', 'poutcome']

# columnas con números
columnas_numericas = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# variables de entrada (X) y salida (y)
X = data[columnas_categoricas + columnas_numericas]
y = data['deposit'].map({'yes': 1, 'no': 0})  # convertir a 0 y 1

# transformamos texto a números y escalamos los valores numéricos
transformador = ColumnTransformer([
    ('categoricas', OneHotEncoder(handle_unknown='ignore'), columnas_categoricas),
    ('numericas', StandardScaler(), columnas_numericas)
])

X = transformador.fit_transform(X)

print("datos procesados:", X.shape)

# aca lo que vamos a hacer es preprocesar por asi decirlo los datos, para luego poder usarlos en el modelo de red neuronal. Y gracias a un transformador aplicamos one hot encoding y, vamos a convertir las variables categóricas en numéricas.

In [ ]:
# separamos los datos en un conjunto de entrenamiento, validación y test

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=0)

print("entrenamiento:", X_train.shape)
print("validación:", X_val.shape)
print("test:", X_test.shape)

# aca dividimos entre train y test, asi vamos a lograr tener un modelo que no overfitee y pueda generalizar bien, cuando se encuentre con datos nuevos, asi logrando un buen desempeño en ambas etapas, no solo en el entrenamiento. Lograndoi que haya digamos un balance.

In [ ]:
# primer modelo de todos, el modelo A

modeloA = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloA.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# entrenamos el modelo
histA = modeloA.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# evaluamos el modelo 
val_lossA, val_accA = modeloA.evaluate(X_val, y_val, verbose=0)
print("modelo A - accuracy validación:", val_accA)

# aca aparece el priemro modelo de red neuronal. Este en si no es un modelo con mucha cioencia por asi decirlo, pero nos sirve ya que cuenta con dos capas ocultas, asi permitiendonos ver, cuan bien va a funcionar este modelo de red neuronal, basico pero dentro de tdoo bastante eficiente.
# vamos a usar relu, y la funcion sigmoidea al final, ya que es un problema de clasificacion binaria.

C:\Users\Famil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


modelo A - accuracy validación: 0.8387818932533264


In [ ]:
# segundo modelo de todos, el modelo B

modeloB = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloB.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# entrenamos el modelo
histB = modeloB.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# evaluamos el modelo 
val_lossB, val_accB = modeloB.evaluate(X_val, y_val, verbose=0)
print("modelo B - accuracy validación:", val_accB)

# aca esta el segundo modelo de red neuronal. Este modelo en si, es mas complejo, ya que cuenta con 3 capaz ocultas, con mas neuronas, que en el anterior modelo. Ambos tienen 20 epoichs, algo que puede modificar mucho el resultado, si no lo ajustamos de manera adecuada.
#  de nuevo vamos a usar relu, y la funcion sigmoidea al final, ya que como dijimos antes, es un problema de clasificacion binaria.

modelo B - accuracy validación: 0.8213165998458862


In [ ]:
# tercer modelo de todos, el modelo C

modeloC = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloC.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# entrenamos el modelo
histC = modeloC.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# evaluamos el modelo
val_lossC, val_accC = modeloC.evaluate(X_val, y_val, verbose=0)
print("modelo C - accuracy validación:", val_accC)

# a primera vista, este modelo puede parecer muy similar al primero, pero al diferencia mas importante, es que la cantidad de neuronas por capa oculta cambia, en este es bastante menor, algo que cambia los resultados. Asi siendo el mejor modelo de todos los que probamos. Tambien este es el que mas rapido entrena, gracias a su menor complejidad digamos. 

modelo C - accuracy validación: 0.8428123593330383


In [ ]:
# elegimos el mejor modelo y lo probamos en el test

# vemos cuál tuvo mejor accuracy
mejor_modelo = None
mejor_acc = 0

if val_accA > mejor_acc:
    mejor_acc = val_accA
    mejor_modelo = modeloA
if val_accB > mejor_acc:
    mejor_acc = val_accB
    mejor_modelo = modeloB
if val_accC > mejor_acc:
    mejor_acc = val_accC
    mejor_modelo = modeloC

# comparamos la accuracy de los tres modelos, y seleccionamos el mejor de todos. 

print("mejor modelo elegido con accuracy:", mejor_acc)

# entrenamos este modelo con el train mas el validation.
mejor_modelo.fit(X_train_full, y_train_full, epochs=20, verbose=0)

# evaluamos en el test
test_loss, test_acc = mejor_modelo.evaluate(X_test, y_test, verbose=0)
print("accuracy final en test:", test_acc)

# por ultimo evaluamos el mejor modelo en el test, para ver que tan bien puede generalizar. Vemos que el tercer modeloe s el mejor, y logra una accuracy bastante buena. Poco menor que la accxuracy que tenia en el train, pero aun asi muy buena.

mejor modelo elegido con accuracy: 0.8428123593330383
accuracy final en test: 0.8392297625541687


In [ ]:
# en si como conclusion final, creemos que estos resultyados fueron mejores. Ya que  al probar con distintos tipos de modelos de redes neuronales, creemos que el mejor, dentreo de todo fue un modelo poco complejo. Asi logrando ser ejecutado rapidamente.
# ademas de que en nuestra opinion a medida de que fuimos haciendo el tp, nos parecio mas interesante usa este tipo de modelos, ya que la estructura para escreiubirlo nos poarece mas amigable digamos, y a pesar de que tuvimos que usar Blackbox AI al principio para entender como funcionaba realmente este modelo (no para escribir el codigo), nos termino pareciendo mejor.
# es por eso que consideramos que estos resultados, en accuracy ya de por si fueron mejores, pero creemos que el hecho de que hayamos usado redes neuronales fue mejor, por como es la estructura de el codigo.